In [12]:
import requests

In [13]:
def body(url):
    # This function will print the content of the body.
    response = requests.get(url)
    html_content = response.text
    texts = []
    start_index = html_content.find("<body")

    if start_index != -1:
        start_index = html_content.find(">", start_index) + 1
        while start_index != -1:
            end_index = html_content.find("<", start_index)
            if end_index != -1:
                text = html_content[start_index:end_index].strip()
                if text: #text.isalnum()
                    texts.append(text)
                start_index = html_content.find(">", end_index) + 1
            else:
                break
    # This will remove the unwanted things from the texts and add to the result.
    result = ''
    for i in texts:
        if (i[0] not in ['.', ',','', ' ', '(', ')', '[', ']', '{', '}', '@', "'", '&', '#', '^']):
            if i[0:3] not in ['-->', 'htt', 'jQu']:
                # if i.isalnum():
                    result = result + i + ' '
            
    return result

In [14]:
def generate_ngrams_and_frequency(web_texts, n):
    # This will lower the 
    word_list = [word.lower() for word in web_texts.split()]

    ngrams = {}
    for i in range(len(word_list) - n + 1):
        ngram = tuple(word_list[i:i + n])
        if ngram in ngrams:
            ngrams[ngram] += 1
        else:
            ngrams[ngram] = 1
    return ngrams

In [15]:
def polynomial_hash_64bit(words_freq):

    hashes =  words_freq

    p = 53
    m = 2**64
    
    for tuple_of_words in hashes:
        hash_val = 0
        for word in tuple_of_words:
            for i, char in enumerate(word):
                hash_val += ord(char) * (p ** i)
        
        b = bin(hash_val % m)
        x = (64 - len(b) + 2) * "0" + b[2:]
        hashes[tuple_of_words] = x
    
    return hashes

In [17]:
def compute_simhash_for_doc(ngrams):
    # Combine word hashes into a single Simhash value
    final_vector = ''
    for i in range(64):
        vec = ''
        for word_hash in ngrams:
            vec = vec + ngrams[word_hash][i]
        
        if vec.count('0') < vec.count('1'):
            final_vector = final_vector + '1'
        else:
            final_vector = final_vector + '0'

    return final_vector

In [18]:
def compare_two_doc():
    doc_url_1 = input("Enter Url 1 :")
    doc_url_2 = input("Enter Url 2 :")
    n = int(input("Enter n-gram value :"))

    web_texts_1 = body(doc_url_1)
    web_texts_2 = body(doc_url_2)

    f1 = generate_ngrams_and_frequency(web_texts_1, n)
    h1 = polynomial_hash_64bit(f1)
    v1 = compute_simhash_for_doc(h1)
    print(v1)

    f2 = generate_ngrams_and_frequency(web_texts_2, n)
    h2 = polynomial_hash_64bit(f2)
    v2 = compute_simhash_for_doc(h2)
    print(v2)

    if len(v1) != len(v2):
        raise ValueError("Lengths of v1 and v2 must be the same")

    total_bits = len(v1)
    matching_bits = sum(1 for bit1, bit2 in zip(v1, v2) if bit1 == bit2)

    match_percentage = (matching_bits / total_bits) * 100

    return match_percentage

In [19]:
# a = https://en.wikipedia.org/wiki/Machine_learning
# b = https://en.wikipedia.org/wiki/Artificial_intelligence
# c = https://timesofindia.indiatimes.com/india/pm-modi-chairs-brainstorming-session-for-viksit-bharat-2047-heres-what-was-discussed/articleshow/108183149.cms

compare_two_doc()

0000000000000000000000000000100000110101101100100110101010000111
0000000000000000000000000000000000100001001011000000010100100111


75.0